## Sentiment data

### Quick examples

In [9]:
import pandas as pd

In [10]:
en_sent_train = pd.read_csv("../data/sentiment/en/train.csv", header=None)
en_sent_train.columns = ["Sentiment", "Review"]

In [11]:
en_sent_train

,Sentiment,Review
0,1,The Rock is destined to be the st Century 's n...
1,1,The gorgeously elaborate continuation of `` Th...
2,1,Singer\/composer Bryan Adams contributes a sle...
3,1,Yet the act is still charming here .
4,1,Whether or not you 're enlightened by any of D...
...,...,...
6915,0,A real snooze .
6916,0,No surprises .
6917,1,We 've seen the hippie-turned-yuppie plot befo...
6918,0,Her fans walked out muttering words like `` ho...


In [4]:
print(en_sent_train["Review"].sample().values[0])

The concept is a hoot .


In [1]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
example = tokenizer("A house is on fire")
example

{'input_ids': [101, 138, 13215, 10124, 10135, 13559, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenizer.decode(example["input_ids"])

'[CLS] A house is on fire [SEP]'

In [7]:
example = tokenizer("房子著火了")
example

{'input_ids': [101, 3998, 3350, 6827, 5257, 2146, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [8]:
tokenizer.decode(example["input_ids"])

'[CLS] 房 子 著 火 了 [SEP]'

In [14]:
[len(x) for x in tokenizer(["aasd aa", "sad aa"])["input_ids"]]

[5, 4]

In [26]:
print(tokenizer(en_sent_train["Review"].values[:3].tolist())["input_ids"])

[[101, 10117, 12158, 10124, 10139, 28830, 10162, 10114, 10347, 10105, 28780, 14262, 112, 187, 10751, 100, 100, 42154, 112, 112, 10111, 10189, 10261, 112, 187, 19090, 10114, 13086, 169, 32650, 25297, 13246, 25722, 11084, 17733, 39050, 12061, 25749, 117, 10766, 118, 101989, 71585, 11474, 104698, 10112, 10345, 17569, 42795, 10161, 119, 102], [101, 10117, 11783, 28163, 47173, 90052, 101576, 11046, 10108, 100, 100, 10117, 13440, 10108, 10105, 71324, 112, 112, 15633, 18583, 10124, 10380, 42126, 10189, 169, 41278, 10108, 21296, 10944, 10472, 10840, 10112, 64207, 10454, 30458, 11170, 118, 17556, 165, 120, 12461, 10979, 13169, 112, 187, 26105, 30237, 10108, 147, 119, 155, 119, 155, 119, 45719, 112, 187, 15441, 118, 39189, 119, 102], [101, 29842, 165, 120, 26474, 24510, 16955, 72484, 10107, 169, 38523, 26127, 10108, 15457, 118, 118, 169, 13824, 23078, 27922, 117, 169, 13824, 10798, 26097, 26391, 13923, 11942, 10114, 10105, 13617, 118, 118, 10473, 10105, 21047, 62432, 88134, 32083, 10107, 10105, 

### Stats

In [64]:
import os
from tqdm.notebook import tqdm

In [65]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [66]:
def stats(path):
    langs = os.listdir(path)
    stats = pd.DataFrame()

    for lang in tqdm(langs):
        path_extended = path + lang + "/"
        
        # Load data
        train = pd.read_csv(path_extended + "train.csv", encoding='utf-8', header=None)
        train.columns = ["Sentiment", "Review"]
        dev = pd.read_csv(path_extended + "dev.csv", encoding='utf-8', header=None)
        dev.columns = ["Sentiment", "Review"]
        test = pd.read_csv(path_extended + "test.csv", encoding='utf-8', header=None)
        test.columns = ["Sentiment", "Review"]
        
        # Tokenize
        n_tokens = []
        for dataset in [train, dev, test]:
            tokenized = tokenizer(dataset["Review"].values.tolist())["input_ids"]
            n_tokens += [len(x) for x in tokenized]
        avg_tokens = sum(n_tokens) / (train.shape[0] + dev.shape[0] + test.shape[0])
        
        stats = pd.concat([stats, pd.DataFrame({"lang": [lang],
                                                "train": [train.shape[0]],
                                                "dev": [dev.shape[0]],
                                                "test": [test.shape[0]],
                                                "avg_tokens": [avg_tokens]})])
    return stats.reset_index(drop=True)

In [67]:
stats_sentiment = stats("../data/sentiment/")

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2157 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (945 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (912 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2936 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (609 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1018 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (820 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1278 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (699 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1929 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (769 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1061 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1466 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1376 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1041 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1490 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1153 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1088 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1086 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1044 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1209 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1390 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (1108 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1291 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1516 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1038 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1199 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (845 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (895 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (648 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1265 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [68]:
stats_sentiment

,lang,train,dev,test,avg_tokens
0,ar,35841,5134,10076,137.562438
1,bg,6208,902,1837,18.213479
2,en,6920,872,1821,27.672735
3,eu,789,113,227,21.245350
4,fi,1355,199,397,93.313685
5,he,6207,904,1771,34.967350
6,hr,1507,214,437,54.661724
7,ko,12312,2532,5072,45.581141
8,ru,2938,424,867,1083.892173
9,sl,3724,532,1064,33.289474


Word count vs number of tokens

In [46]:
en_sent_train["Review"].apply(lambda x: len(x.split())).mean()

19.266473988439305

In [47]:
ar_sent_train = pd.read_csv("../data/sentiment/ar/train.csv", header=None)
ar_sent_train.columns = ["Sentiment", "Review"]
ar_sent_train["Review"].apply(lambda x: len(x.split())).mean()

61.42205853631316

In [51]:
len(ar_sent_train["Review"][0].split())

106

In [55]:
len(tokenizer(ar_sent_train["Review"][0])["input_ids"])

271

In [56]:
tr_sent_train = pd.read_csv("../data/sentiment/tr/train.csv", header=None)
tr_sent_train.columns = ["Sentiment", "Review"]
tr_sent_train["Review"].apply(lambda x: len(x.split())).mean()

613.0257575757576

### Output stats

In [57]:
stats_sentiment.at[9, "lang"] = "sk"

In [58]:
lang_codes = pd.read_excel("lang_codes.xlsx", header=0)

In [59]:
lang_codes.sample(5)

,ISO 639-2 Code,ISO 639-1 Code,English name of Language,French name of Language,German name of Language
293,lun,NaN,Lunda,lunda,Lunda-Sprache
331,mri (T),NaN,NaN,NaN,NaN
241,kab,NaN,Kabyle,kabyle,Kabylisch
388,peo,NaN,"Persian, Old (ca.600-400 B.C.)","perse, vieux (ca. 600-400 av. J.-C.)",Altpersisch
12,alb (B),sq,Albanian,albanais,Albanisch


In [60]:
def write_table(filename, stats_table, lang_codes):
    file = open(filename, "r")
    output = ""

    for line in file.readlines():
        lang_name = line.split("&")[1].strip()
        lang_code = lang_codes["ISO 639-1 Code"][lang_codes["English name of Language"] == lang_name].values[0]
        df = stats_table[stats_table["lang"] == lang_code]

        if not(df.empty):
            split_line = line.split("\\")
            start = split_line[0] + "\\" + "&".join(split_line[1].split("&")[:2])
            end = r"\\" + "".join(split_line[2:])
            add = df.values[0, 1:].astype("str")
            new_line = start + "& " + " & ".join(add) + end
        else:
            new_line = line

        output += new_line

    file.close()
    return output

In [61]:
sentiment_table = write_table("sentiment_table.txt", stats_sentiment.round(1), lang_codes)

In [62]:
print(sentiment_table)

    \fusional{Fusional}  & Bulgarian & 6208 & 902 & 1837 & 18.2\\ % & url{http://bkapukaranov.github.io/}
    \fusional{Fusional} & English & 6920 & 872 & 1821 & 27.7\\
    \fusional{Fusional}  & Russian & 2938 & 424 & 867 & 1083.9\\ %& url{https://www.kaggle.com/c/6766/download-all}
    \fusional{Fusional} & Slovak & 3724 & 532 & 1064 & 33.3\\ %& url{https://www.aclweb.org/anthology/W19-3716} 
    \fusional{Fusional}  & Croatian & 1507 & 214 & 437 & 54.7\\ %& url{http://takelab.fer.hr/data/cropinion/} 
    \isolating{Isolating} & Chinese & 19292 & 2756 & 5513 & 80.2\\ %& url{http://www.informatics.sussex.ac.uk/users/tz21/it168test.zip} 
    \isolating{Isolating} & Vietnamese & 2384 & 331 & 685 & 36.6\\ %& url{http://vlsp.org.vn/resources-vlsp2016} 
    \isolating{Isolating} & Thai & 8103 & 1154 & 2344 & 53.9\\ %https://github.com/PyThaiNLP/wisesight-sentiment
    \agglutinative{Agglutinative} & Finnish & 1355 & 199 & 397 & 93.3\\ %& url{https://dspace.cc.tut.fi/dpub/bitstream/handle/1

## POS data

In [2]:
import pandas as pd
from conllu import parse

### Quick examples

In [13]:
en_pos = open("../data/ud/en/en_pud-ud-test.conllu", "r", encoding="utf-8")

In [14]:
en_pos = en_pos.read()

In [19]:
print(en_pos[:500])

# newdoc id = n01001
# sent_id = n01001011
# text = “While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.
1	“	“	PUNCT	``	_	20	punct	20:punct	SpaceAfter=No
2	While	while	SCONJ	IN	_	9	mark	9:mark	_
3	much	much	ADJ	JJ	Degree=Pos	9	nsubj	9:nsubj	_
4	of	of	ADP	IN	_	7	case	7:case	_
5	the	the	DET	DT	Definite=Def|PronType=Art	7	det	7:det	_
6	digital	digital	ADJ	JJ	Degree=Po


In [17]:
sentences = parse(en_pos)

Number of examples (sentences)

In [24]:
len(sentences)

1000

In [20]:
sentences[0]

TokenList<“, While, much, of, the, digital, transition, is, unprecedented, in, the, United, States, ,, the, peaceful, transition, of, power, is, not, ,, ”, Obama, special, assistant, Kori, Schulman, wrote, in, a, blog, post, Monday, .>

Number of tokens in a sentence

In [28]:
len(sentences[0])

35

In [30]:
sentences[0][1]

Token([('id', 2),
       ('form', 'While'),
       ('lemma', 'while'),
       ('upos', 'SCONJ'),
       ('xpos', 'IN'),
       ('feats', None),
       ('head', 9),
       ('deprel', 'mark'),
       ('deps', [('mark', 9)]),
       ('misc', None)])

In [32]:
[x["upos"] for x in sentences[0]][:10]

['PUNCT', 'SCONJ', 'ADJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'AUX', 'ADJ', 'ADP']